In [1]:
import seaborn as sns

import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

from sklearn import set_config
set_config(transform_output="pandas")

# Set random seed 
RSEED = 42

warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv("data/cleaned_data.csv")

In [5]:
df.head()

,Unnamed: 0,date_caught,capture_site,tag_2,ccl_cm,ccw_cm,weight_kg,status,release_site,date_released,...,landing_site_LandingSite_CaptureSiteCategory_3,landing_site_LandingSite_CaptureSiteCategory_4,species_Species_0,species_Species_1,species_Species_2,species_Species_3,species_Species_4,species_Species_5,species_Species_6,species_Species_7
0,0,2000-12-22,CaptureSite_0,0,64.70,62.60,8.5,429,237,2000-12-22 00:00:00,...,False,False,False,False,False,False,False,False,True,False
1,1,2001-10-28,CaptureSite_0,0,35.85,31.35,8.5,429,250,2001-10-28 00:00:00,...,False,False,False,False,False,False,False,False,True,False
2,2,2001-11-01,CaptureSite_0,0,51.80,49.20,8.5,429,237,2001-01-11 00:00:00,...,False,False,False,False,False,False,False,True,False,False
3,3,2002-03-11,CaptureSite_0,0,60.50,59.00,8.5,429,237,2002-11-03 00:00:00,...,False,False,False,False,False,False,False,False,True,False
4,4,2002-08-08,CaptureSite_0,0,34.70,33.00,8.5,429,250,2002-08-08 00:00:00,...,False,False,False,False,False,False,False,True,False,False


In [3]:
capture_site_category = pd.read_csv('data/CaptureSite_category.csv')
sample_submission = pd.read_csv('data/Sample_sub.csv')

In [6]:
pd.set_option('display.max_colwidth', None)

In [36]:
df.columns

Index(['Unnamed: 0', 'date_caught', 'capture_site', 'tag_2', 'ccl_cm',
       'ccw_cm', 'weight_kg', 'status', 'release_site', 'date_released',
       'capture_method_beached', 'capture_method_by hand',
       'capture_method_collected floater', 'capture_method_fish trap',
       'capture_method_jarife', 'capture_method_longline',
       'capture_method_net', 'capture_method_not_recorded',
       'capture_method_speargun', 'capture_method_stranded',
       'capture_method_uzio', 'foraging_ground_0', 'foraging_ground_1',
       'landing_site_LandingSite_CaptureSiteCategory_0',
       'landing_site_LandingSite_CaptureSiteCategory_1',
       'landing_site_LandingSite_CaptureSiteCategory_2',
       'landing_site_LandingSite_CaptureSiteCategory_3',
       'landing_site_LandingSite_CaptureSiteCategory_4', 'species_Species_0',
       'species_Species_1', 'species_Species_2', 'species_Species_3',
       'species_Species_4', 'species_Species_5', 'species_Species_6',
       'species_Species_7', 

In [39]:
capture_site_category.head()

,capture_site,CaptureSiteCategory,Type
0,CaptureSite_0,CaptureSiteCategory_2,Type_1
1,CaptureSite_1,CaptureSiteCategory_2,Type_1
2,CaptureSite_10,CaptureSiteCategory_2,Type_1
3,CaptureSite_11,CaptureSiteCategory_0,Type_0
4,CaptureSite_12,CaptureSiteCategory_2,Type_1


In [37]:
#format column names
capture_site_category = capture_site_category.rename({'CaptureSite':'capture_site'},axis=1)

In [38]:
capture_site_category.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   capture_site         29 non-null     category
 1   CaptureSiteCategory  29 non-null     category
 2   Type                 29 non-null     category
dtypes: category(3)
memory usage: 1.8 KB


In [11]:
categorical_columns = ['CaptureSite', 'CaptureSiteCategory', 'Type'
                       ]

for col in categorical_columns:
    capture_site_category[col] = capture_site_category[col].astype('category')

In [12]:
capture_site_category.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   CaptureSite          29 non-null     category
 1   CaptureSiteCategory  29 non-null     category
 2   Type                 29 non-null     category
dtypes: category(3)
memory usage: 1.8 KB


In [13]:
sample_submission.head()

,ID,Capture_Number
0,CaptureSite_0_201901,7
1,CaptureSite_0_201902,1
2,CaptureSite_0_201903,5
3,CaptureSite_0_201904,2
4,CaptureSite_0_201905,3


In [14]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1276 entries, 0 to 1275
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              1276 non-null   object
 1   Capture_Number  1276 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 20.1+ KB


In [15]:
categorical_column = ['ID'
                       ]

for col in categorical_column:
    sample_submission[col] = sample_submission[col].astype('category')

In [16]:
# Data wrangling 
sample_submission["year_woy"]=(sample_submission.ID.apply(lambda x: x.split("_")[-1])).astype(int)
sample_submission["CaptureSite"]=sample_submission.ID.apply(lambda x: ("_").join(x.split("_")[0:-1]))
sample_submission.head()

,ID,Capture_Number,year_woy,CaptureSite
0,CaptureSite_0_201901,7,201901,CaptureSite_0
1,CaptureSite_0_201902,1,201902,CaptureSite_0
2,CaptureSite_0_201903,5,201903,CaptureSite_0
3,CaptureSite_0_201904,2,201904,CaptureSite_0
4,CaptureSite_0_201905,3,201905,CaptureSite_0


In [19]:
df.columns

Index(['Unnamed: 0', 'date_caught', 'capture_site', 'tag_2', 'ccl_cm',
       'ccw_cm', 'weight_kg', 'status', 'release_site', 'date_released',
       'capture_method_beached', 'capture_method_by hand',
       'capture_method_collected floater', 'capture_method_fish trap',
       'capture_method_jarife', 'capture_method_longline',
       'capture_method_net', 'capture_method_not_recorded',
       'capture_method_speargun', 'capture_method_stranded',
       'capture_method_uzio', 'foraging_ground_0', 'foraging_ground_1',
       'landing_site_LandingSite_CaptureSiteCategory_0',
       'landing_site_LandingSite_CaptureSiteCategory_1',
       'landing_site_LandingSite_CaptureSiteCategory_2',
       'landing_site_LandingSite_CaptureSiteCategory_3',
       'landing_site_LandingSite_CaptureSiteCategory_4', 'species_Species_0',
       'species_Species_1', 'species_Species_2', 'species_Species_3',
       'species_Species_4', 'species_Species_5', 'species_Species_6',
       'species_Species_7'],

In [22]:
df.head(5)

,Unnamed: 0,date_caught,capture_site,tag_2,ccl_cm,ccw_cm,weight_kg,status,release_site,date_released,...,landing_site_LandingSite_CaptureSiteCategory_4,species_Species_0,species_Species_1,species_Species_2,species_Species_3,species_Species_4,species_Species_5,species_Species_6,species_Species_7,year
0,0,2000-12-22,CaptureSite_0,0,64.70,62.60,8.5,429,237,2000-12-22 00:00:00,...,False,False,False,False,False,False,False,True,False,2000
1,1,2001-10-28,CaptureSite_0,0,35.85,31.35,8.5,429,250,2001-10-28 00:00:00,...,False,False,False,False,False,False,False,True,False,2001
2,2,2001-11-01,CaptureSite_0,0,51.80,49.20,8.5,429,237,2001-01-11 00:00:00,...,False,False,False,False,False,False,True,False,False,2001
3,3,2002-03-11,CaptureSite_0,0,60.50,59.00,8.5,429,237,2002-11-03 00:00:00,...,False,False,False,False,False,False,False,True,False,2002
4,4,2002-08-08,CaptureSite_0,0,34.70,33.00,8.5,429,250,2002-08-08 00:00:00,...,False,False,False,False,False,False,True,False,False,2002


In [24]:
df["date_caught"] = pd.to_datetime(df["date_caught"])
df["year"] = df["date_caught"].dt.year
df["week_of_year"] = df["date_caught"].dt.isocalendar().week  # Using isocalendar() method
df["year_woy"] = df["year"] * 100 + df["week_of_year"]
df.head()



,Unnamed: 0,date_caught,capture_site,tag_2,ccl_cm,ccw_cm,weight_kg,status,release_site,date_released,...,species_Species_1,species_Species_2,species_Species_3,species_Species_4,species_Species_5,species_Species_6,species_Species_7,year,week_of_year,year_woy
0,0,2000-12-22,CaptureSite_0,0,64.70,62.60,8.5,429,237,2000-12-22 00:00:00,...,False,False,False,False,False,True,False,2000,51,200051
1,1,2001-10-28,CaptureSite_0,0,35.85,31.35,8.5,429,250,2001-10-28 00:00:00,...,False,False,False,False,False,True,False,2001,43,200143
2,2,2001-11-01,CaptureSite_0,0,51.80,49.20,8.5,429,237,2001-01-11 00:00:00,...,False,False,False,False,True,False,False,2001,44,200144
3,3,2002-03-11,CaptureSite_0,0,60.50,59.00,8.5,429,237,2002-11-03 00:00:00,...,False,False,False,False,False,True,False,2002,11,200211
4,4,2002-08-08,CaptureSite_0,0,34.70,33.00,8.5,429,250,2002-08-08 00:00:00,...,False,False,False,False,True,False,False,2002,32,200232


In [26]:
df.columns

Index(['Unnamed: 0', 'date_caught', 'capture_site', 'tag_2', 'ccl_cm',
       'ccw_cm', 'weight_kg', 'status', 'release_site', 'date_released',
       'capture_method_beached', 'capture_method_by hand',
       'capture_method_collected floater', 'capture_method_fish trap',
       'capture_method_jarife', 'capture_method_longline',
       'capture_method_net', 'capture_method_not_recorded',
       'capture_method_speargun', 'capture_method_stranded',
       'capture_method_uzio', 'foraging_ground_0', 'foraging_ground_1',
       'landing_site_LandingSite_CaptureSiteCategory_0',
       'landing_site_LandingSite_CaptureSiteCategory_1',
       'landing_site_LandingSite_CaptureSiteCategory_2',
       'landing_site_LandingSite_CaptureSiteCategory_3',
       'landing_site_LandingSite_CaptureSiteCategory_4', 'species_Species_0',
       'species_Species_1', 'species_Species_2', 'species_Species_3',
       'species_Species_4', 'species_Species_5', 'species_Species_6',
       'species_Species_7', 

In [29]:
sample_submission.head()

,ID,Capture_Number,year_woy,CaptureSite
0,CaptureSite_0_201901,7,201901,CaptureSite_0
1,CaptureSite_0_201902,1,201902,CaptureSite_0
2,CaptureSite_0_201903,5,201903,CaptureSite_0
3,CaptureSite_0_201904,2,201904,CaptureSite_0
4,CaptureSite_0_201905,3,201905,CaptureSite_0


In [30]:
#format column names
sample_submission = sample_submission.rename({'Capture_Number':'capture_number', 'CaptureSite':'capture_site'},axis=1)

In [31]:
sample_submission.head()

,ID,capture_number,year_woy,capture_site
0,CaptureSite_0_201901,7,201901,CaptureSite_0
1,CaptureSite_0_201902,1,201902,CaptureSite_0
2,CaptureSite_0_201903,5,201903,CaptureSite_0
3,CaptureSite_0_201904,2,201904,CaptureSite_0
4,CaptureSite_0_201905,3,201905,CaptureSite_0


In [32]:

# Grouping the data by capture site and taking the minimum week of year
keys=pd.concat([df[["year_woy","capture_site"]],sample_submission[["year_woy","capture_site"]]])
capture_site_min_year_woy=keys.groupby("capture_site").year_woy.min().rename("year_woy").reset_index()

In [57]:
df.columns

Index(['Unnamed: 0', 'date_caught', 'capture_site', 'tag_2', 'ccl_cm',
       'ccw_cm', 'weight_kg', 'status', 'release_site', 'date_released',
       'capture_method_beached', 'capture_method_by hand',
       'capture_method_collected floater', 'capture_method_fish trap',
       'capture_method_jarife', 'capture_method_longline',
       'capture_method_net', 'capture_method_not_recorded',
       'capture_method_speargun', 'capture_method_stranded',
       'capture_method_uzio', 'foraging_ground_0', 'foraging_ground_1',
       'landing_site_LandingSite_CaptureSiteCategory_0',
       'landing_site_LandingSite_CaptureSiteCategory_1',
       'landing_site_LandingSite_CaptureSiteCategory_2',
       'landing_site_LandingSite_CaptureSiteCategory_3',
       'landing_site_LandingSite_CaptureSiteCategory_4', 'species_Species_0',
       'species_Species_1', 'species_Species_2', 'species_Species_3',
       'species_Species_4', 'species_Species_5', 'species_Species_6',
       'species_Species_7', 

In [59]:
# Extracting the target variable from the dataset
target=df.groupby(["year_woy","capture_site"]).capture_site.count().rename("Capture_Number").reset_index()

In [60]:
target

,year_woy,capture_site,Capture_Number
0,199816,CaptureSite_27,1
1,199828,CaptureSite_11,1
2,199832,CaptureSite_11,1
3,199832,CaptureSite_27,1
4,199839,CaptureSite_11,2
...,...,...,...
7952,201852,CaptureSite_23,1
7953,201852,CaptureSite_25,4
7954,201852,CaptureSite_3,1
7955,201852,CaptureSite_7,1


In [61]:
target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7957 entries, 0 to 7956
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year_woy        7957 non-null   Int64 
 1   capture_site    7957 non-null   object
 2   Capture_Number  7957 non-null   int64 
dtypes: Int64(1), int64(1), object(1)
memory usage: 194.4+ KB


In [62]:
target.isnull().sum()

year_woy          0
capture_site      0
Capture_Number    0
dtype: int64

In [40]:
capture_site_category.head()

,capture_site,CaptureSiteCategory,Type
0,CaptureSite_0,CaptureSiteCategory_2,Type_1
1,CaptureSite_1,CaptureSiteCategory_2,Type_1
2,CaptureSite_10,CaptureSiteCategory_2,Type_1
3,CaptureSite_11,CaptureSiteCategory_0,Type_0
4,CaptureSite_12,CaptureSiteCategory_2,Type_1


In [35]:
df.head(5)

,Unnamed: 0,date_caught,capture_site,tag_2,ccl_cm,ccw_cm,weight_kg,status,release_site,date_released,...,species_Species_1,species_Species_2,species_Species_3,species_Species_4,species_Species_5,species_Species_6,species_Species_7,year,week_of_year,year_woy
0,0,2000-12-22,CaptureSite_0,0,64.70,62.60,8.5,429,237,2000-12-22 00:00:00,...,False,False,False,False,False,True,False,2000,51,200051
1,1,2001-10-28,CaptureSite_0,0,35.85,31.35,8.5,429,250,2001-10-28 00:00:00,...,False,False,False,False,False,True,False,2001,43,200143
2,2,2001-11-01,CaptureSite_0,0,51.80,49.20,8.5,429,237,2001-01-11 00:00:00,...,False,False,False,False,True,False,False,2001,44,200144
3,3,2002-03-11,CaptureSite_0,0,60.50,59.00,8.5,429,237,2002-11-03 00:00:00,...,False,False,False,False,False,True,False,2002,11,200211
4,4,2002-08-08,CaptureSite_0,0,34.70,33.00,8.5,429,250,2002-08-08 00:00:00,...,False,False,False,False,True,False,False,2002,32,200232


In [43]:
df_1 = pd.merge (left=df, right=capture_site_category, left_on ='capture_site', right_on = 'capture_site', how='left')

In [46]:
df_1.head()

,Unnamed: 0,date_caught,capture_site,tag_2,ccl_cm,ccw_cm,weight_kg,status,release_site,date_released,...,species_Species_3,species_Species_4,species_Species_5,species_Species_6,species_Species_7,year,week_of_year,year_woy,CaptureSiteCategory,Type
0,0,2000-12-22,CaptureSite_0,0,64.70,62.60,8.5,429,237,2000-12-22 00:00:00,...,False,False,False,True,False,2000,51,200051,CaptureSiteCategory_2,Type_1
1,1,2001-10-28,CaptureSite_0,0,35.85,31.35,8.5,429,250,2001-10-28 00:00:00,...,False,False,False,True,False,2001,43,200143,CaptureSiteCategory_2,Type_1
2,2,2001-11-01,CaptureSite_0,0,51.80,49.20,8.5,429,237,2001-01-11 00:00:00,...,False,False,True,False,False,2001,44,200144,CaptureSiteCategory_2,Type_1
3,3,2002-03-11,CaptureSite_0,0,60.50,59.00,8.5,429,237,2002-11-03 00:00:00,...,False,False,False,True,False,2002,11,200211,CaptureSiteCategory_2,Type_1
4,4,2002-08-08,CaptureSite_0,0,34.70,33.00,8.5,429,250,2002-08-08 00:00:00,...,False,False,True,False,False,2002,32,200232,CaptureSiteCategory_2,Type_1


In [45]:
df_1.shape

(18062, 41)

In [64]:
final_data=df_1.merge(target,on=["year_woy","capture_site"],how="left")

In [65]:
final_data.head()

,Unnamed: 0,date_caught,capture_site,tag_2,ccl_cm,ccw_cm,weight_kg,status,release_site,date_released,...,species_Species_4,species_Species_5,species_Species_6,species_Species_7,year,week_of_year,year_woy,CaptureSiteCategory,Type,Capture_Number
0,0,2000-12-22,CaptureSite_0,0,64.70,62.60,8.5,429,237,2000-12-22 00:00:00,...,False,False,True,False,2000,51,200051,CaptureSiteCategory_2,Type_1,1
1,1,2001-10-28,CaptureSite_0,0,35.85,31.35,8.5,429,250,2001-10-28 00:00:00,...,False,False,True,False,2001,43,200143,CaptureSiteCategory_2,Type_1,1
2,2,2001-11-01,CaptureSite_0,0,51.80,49.20,8.5,429,237,2001-01-11 00:00:00,...,False,True,False,False,2001,44,200144,CaptureSiteCategory_2,Type_1,1
3,3,2002-03-11,CaptureSite_0,0,60.50,59.00,8.5,429,237,2002-11-03 00:00:00,...,False,False,True,False,2002,11,200211,CaptureSiteCategory_2,Type_1,1
4,4,2002-08-08,CaptureSite_0,0,34.70,33.00,8.5,429,250,2002-08-08 00:00:00,...,False,True,False,False,2002,32,200232,CaptureSiteCategory_2,Type_1,2


In [68]:
final_data.isnull().sum()

Unnamed: 0                                        0
date_caught                                       0
capture_site                                      0
tag_2                                             0
ccl_cm                                            0
ccw_cm                                            0
weight_kg                                         0
status                                            0
release_site                                      0
date_released                                     0
capture_method_beached                            0
capture_method_by hand                            0
capture_method_collected floater                  0
capture_method_fish trap                          0
capture_method_jarife                             0
capture_method_longline                           0
capture_method_net                                0
capture_method_not_recorded                       0
capture_method_speargun                           0
capture_meth

In [70]:
final_data['Capture_Number'].value_counts()

Capture_Number
1     4136
2     3336
3     2577
4     1768
5     1290
7     1071
6     1008
8      584
9      531
10     450
11     308
14     210
12     204
13     182
15      90
16      80
17      51
35      35
28      28
23      23
22      22
21      21
20      20
19      19
18      18
Name: count, dtype: int64

In [82]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18062 entries, 0 to 18061
Data columns (total 42 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   Unnamed: 0                                      18062 non-null  int64         
 1   date_caught                                     18062 non-null  datetime64[ns]
 2   capture_site                                    18062 non-null  int32         
 3   tag_2                                           18062 non-null  int64         
 4   ccl_cm                                          18062 non-null  float64       
 5   ccw_cm                                          18062 non-null  float64       
 6   weight_kg                                       18062 non-null  float64       
 7   status                                          18062 non-null  int64         
 8   release_site                                  

In [80]:
final_data['capture_site'].nunique()

29

In [81]:
final_data['capture_site'] = LabelEncoder().fit_transform(final_data['capture_site'])

In [85]:
final_data['CaptureSiteCategory'].nunique()

5

In [86]:
final_data['CaptureSiteCategory'] = LabelEncoder().fit_transform(final_data['CaptureSiteCategory'])

In [83]:
final_data['Type'].nunique()

2

In [87]:
final_data['Type'] = LabelEncoder().fit_transform(final_data['Type'])

In [94]:
del final_data['date_released'] 

In [73]:
final_data['Capture_Number']

0        1
1        1
2        1
3        1
4        2
        ..
18057    5
18058    5
18059    3
18060    3
18061    3
Name: Capture_Number, Length: 18062, dtype: int64

In [95]:
# Select X and y features
X = final_data.drop(['Capture_Number'], axis = 1)
y = final_data['Capture_Number']

In [96]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18062 entries, 0 to 18061
Data columns (total 40 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   Unnamed: 0                                      18062 non-null  int64         
 1   date_caught                                     18062 non-null  datetime64[ns]
 2   capture_site                                    18062 non-null  int32         
 3   tag_2                                           18062 non-null  int64         
 4   ccl_cm                                          18062 non-null  float64       
 5   ccw_cm                                          18062 non-null  float64       
 6   weight_kg                                       18062 non-null  float64       
 7   status                                          18062 non-null  int64         
 8   release_site                                  

In [97]:
# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True, stratify=y)

# Check the shape of the data sets
print("X_train:", X_train.shape)  
print("y_train:", y_train.shape)  
print("X_test:", X_test.shape) 
print("y_test:", y_test.shape)   

X_train: (12643, 40)
y_train: (12643,)
X_test: (5419, 40)
y_test: (5419,)
